* https://www.kaggle.com/code/shahules/cord-tools-and-knowledge-graphs
* https://neo4j.com/generativeai/
* https://towardsdatascience.com/create-a-graph-database-in-neo4j-using-python-4172d40f89c4
* https://towardsdatascience.com/extract-knowledge-from-text-end-to-end-information-extraction-pipeline-with-spacy-and-neo4j-502b2b1e0754
* https://neo4j.com/docs/operations-manual/current/docker/introduction/
* https://github.com/JohannesJolkkonen/funktio-ai-samples/blob/main/knowledge-graph-demo/notebook.ipynb / https://www.youtube.com/watch?v=tcHIDCGu6Yw
* https://neo4j.com/product/cypher-graph-query-language/
* https://nanonets.com/blog/langchain/#module-ii-retrieval
* https://python.langchain.com/docs/use_cases/graph/integrations/graph_cypher_qa

new
* https://huggingface.co/distilbert/distilbert-base-multilingual-cased
* https://huggingface.co/cointegrated/rubert-tiny
* https://neo4j.com/docs/cypher-manual/current/indexes/semantic-indexes/full-text-indexes/

# Dependencies

In [1]:
# !ls

In [3]:
# %pip install retry

In [4]:
# !wget https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar

In [5]:
from os import listdir
from os.path import isfile, join

import numpy as np
from numpy.linalg import norm
from enum import Enum
import typing
from abc import ABC
import re

import natasha
from navec import Navec
import transformers
from transformers.pipelines import token_classification
from langchain_text_splitters import RecursiveCharacterTextSplitter
import torch


import matplotlib.pyplot as plt

import gc

import spacy

from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers.pipelines.token_classification import AggregationStrategy

from retry import retry

# Prep

In [7]:
import pandas as pd
import glob
import os

In [8]:
class PrintLogger:
    def debug(self, text):
        print(text)
        # pass
    def info(self, text):
        print(text)
    def warning(self, text):
        print(text)
    def error(self, text):
        print(text)

logger = PrintLogger()
#logging.getLogger(__name__)
# logging.basicConfig(filename='example.log', encoding='utf-8', level=logging.DEBUG)
# logger.debug('debug')

In [9]:
directory = '../../data'

In [10]:
articles_info = pd.read_csv(f'{directory}/articles_info.csv', sep='_')
articles_info.head()

,title,link,hash
0,Музей «Павловские Колтуши» — первый российский...,https://travel.mts.ru/media/guides/101-ideya/m...,6608806741320959874
1,Юсуповский дворец — дом богатейшей семьи Росс...,https://travel.mts.ru/media/guides/101-ideya/y...,4474475373476231224
2,Дом кино: известные здания в фильмах «Брат» и ...,https://realt.onliner.by/2012/10/14/darriuss-27,5158891874358032262
3,Колоннада Исаакиевского собора: Вид с высоты н...,https://travel.mts.ru/media/guides/101-ideya/k...,2528798168100379865
4,Стрелка Васильевского острова: Главная петербу...,https://travel.mts.ru/media/guides/101-ideya/s...,2277906714354252759


In [11]:
list(articles_info['link'])

['https://travel.mts.ru/media/guides/101-ideya/muzej-pavlovskie-koltushi',
 'https://travel.mts.ru/media/guides/101-ideya/yusupovskij-dvorecz',
 'https://realt.onliner.by/2012/10/14/darriuss-27',
 'https://travel.mts.ru/media/guides/101-ideya/kolonnada-isaakievskogo-sobora',
 'https://travel.mts.ru/media/guides/101-ideya/strelka-vasilevskogo-ostrova',
 'https://travel.mts.ru/media/guides/101-ideya/anichkov-most',
 'https://travel.mts.ru/media/guides/101-ideya/podpisnye-izdaniya',
 'https://travel.mts.ru/media/guides/101-ideya/spas-na-krovi',
 'https://travel.mts.ru/media/guides/101-ideya/pavlovskij-pejzazhnyj-park',
 'https://travel.mts.ru/media/guides/101-ideya/sobornaya-mechet-1',
 'https://travel.mts.ru/media/guides/101-ideya/vuoksa-park',
 'https://travel.mts.ru/media/guides/101-ideya/elagin-ostrov',
 'https://travel.mts.ru/media/guides/101-ideya/petropavlovskaya-krepost',
 'https://travel.mts.ru/media/guides/101-ideya/tolstovskij-dom-i-dom-benua',
 'https://peterburg.center/ln/mar

In [12]:
directory

'../../data'

In [13]:
pattern = os.path.join(f'{directory}/', '*.txt')
txt_files = glob.glob(pattern)

In [14]:
txt_files

['../../data/4474475373476231224.txt',
 '../../data/5723206605418886168.txt',
 '../../data/2641653856583702694.txt',
 '../../data/7660736790506138341.txt',
 '../../data/6608806741320959874.txt',
 '../../data/2305003458459233100.txt',
 '../../data/2385853299821716352.txt',
 '../../data/2528798168100379865.txt',
 '../../data/2277906714354252759.txt',
 '../../data/5217320806314180457.txt',
 '../../data/649818539170166932.txt',
 '../../data/5016280246236556815.txt',
 '../../data/8853929468922737102.txt',
 '../../data/6825657549006065031.txt',
 '../../data/4517993868704124781.txt',
 '../../data/153120577918414673.txt',
 '../../data/1323561730236642695.txt',
 '../../data/723584773945224988.txt',
 '../../data/4973173965517374633.txt',
 '../../data/5158891874358032262.txt']

In [15]:
text_data = []
for p in txt_files:
    text = ''
    parts = p.split('/')
    file = parts[3]
    name = file.split('.')[0]
    with open(p, 'r') as f:
        text = f.read()
    text_data.append([text, int(name)])
txt_df = pd.DataFrame(text_data, columns=['text', 'hash'])
txt_df.head()

,text,hash
0,Правильнее всего осматривать Юсуповский дворец...,4474475373476231224
1,Мечеть появилась в городе в 1913 году на деньг...,5723206605418886168
2,Для обустройства своей летней резиденции будущ...,2641653856583702694
3,Павловск Екатерина II подарила своему сыну Пав...,7660736790506138341
4,В конструктивистском здании Научного городка с...,6608806741320959874


In [16]:
articles_df = pd.merge(txt_df, articles_info, on='hash')
articles_df.head()

,text,hash,title,link
0,Правильнее всего осматривать Юсуповский дворец...,4474475373476231224,Юсуповский дворец — дом богатейшей семьи Росс...,https://travel.mts.ru/media/guides/101-ideya/y...
1,Мечеть появилась в городе в 1913 году на деньг...,5723206605418886168,Соборная мечеть — мусульманский храм на Петро...,https://travel.mts.ru/media/guides/101-ideya/s...
2,Для обустройства своей летней резиденции будущ...,2641653856583702694,Павловский пейзажный парк — один из самых бол...,https://travel.mts.ru/media/guides/101-ideya/p...
3,Павловск Екатерина II подарила своему сыну Пав...,7660736790506138341,Большой дворец в Павловске: впечатляющая колле...,https://travel.mts.ru/media/guides/101-ideya/b...
4,В конструктивистском здании Научного городка с...,6608806741320959874,Музей «Павловские Колтуши» — первый российский...,https://travel.mts.ru/media/guides/101-ideya/m...


In [17]:
# def get_top_objects():
#     import gc
#     import sys
#     import pandas as pd
#     # Force a collection to clean up any unreferenced objects
#     gc.collect()
    
#     # Retrieve all objects currently tracked by the garbage collector
#     objects = gc.get_objects()
#     print(f"Total objects: {len(objects)}")
    
#     # Iterate over all objects and print their sizes
#     packs = []
#     for obj in objects:
#         try:
#             # Retrieve the object's name, if possible
#             type_name = type(obj).__name__
#             # name = obj.__name__
#             # Get the size of the object in bytes
#             size = sys.getsizeof(obj)
#             preview = str(obj)[:100]
#             packs.append([type_name, size, preview])
#         except Exception as e:
#             print(f"Error retrieving size for object: {e}")
    
#     mdf = pd.DataFrame(packs, columns =['type', 'size', 'preview'])
#     display(mdf['size'].describe())
#     display(mdf[mdf['size'] > 1_000_000].sort_values(by='size', ascending=False))
    
# get_top_objects()

# NER & RE

## Models

In [20]:
from dataclasses import dataclass, field

In [21]:
@dataclass
class Doc:
    title: str
    text: str
    link: str
    
    @property
    def full_text(self) -> str:
        return self.title + '\n' + self.text
    
@dataclass
class Span:
    text: str
    
@dataclass
class Place:
    text: str
    address: str
    category: str
    city: str
    lat: str
    lon: str
    rating: str
    subcategory: str
    title: str
    object_keys_text: str
    is_modern: str
    has_music: str
    fill_score: str
    is_accessable: str
    review_count: str
    worktime_is_24x7: str
    has_wifi: str
    contact_social_len: str
    phone_len: str
    photos_len: str
    has_website: str
    biased_rating: str
    
@dataclass
class Entity:
    id: int
    
    @property
    def priority(self) -> int:
        return self.id

@dataclass
class Embeded:
    embeddings: list

@dataclass
class Node(Entity):
    relations: list['Relation']
    backtracks: list['Relation']
    
    @property
    def identity(self) -> str | None:
        pass
    
    @identity.setter
    def identity(self, value: str):
        pass
    
RELATION_BASE_PRIORITY = 10_000

@dataclass
class Relation(Entity):
    head: Node
    tail: Node
    type_: str
    
    @property
    def label(self) -> str:
        return self.type_.replace(' ', '_').upper()
    
    def cypher(self) -> tuple[str, dict]:
        return (
            f'MATCH (t:{self.tail.identity_pattern}), (h:{self.head.identity_pattern})'
            f'CREATE (t)-[:{self.label}]->(h)'
        ), {}
    
    @property
    def priority(self) -> int:
        return self.id + RELATION_BASE_PRIORITY

@dataclass
class CypherNode(Node):
    __data_fields__: typing.ClassVar[list[str]]
    __unique_fields__: typing.ClassVar[list[str]]
    
    __lable__: typing.ClassVar[str]
    __variable__: typing.ClassVar[str]
    
    @property
    def variable(self):
        return f'{self.__variable__}{self.id}'
    
    @property
    def identity_pattern(self) -> str:
        return f'{self.__lable__} {{id: {self.id}}}'

    def __build_data(self):
        node_data = {k: v for k, v in self.__dict__.items() if k in self.__data_fields__}
        if not len(node_data):
            return '{ }', dict()
        
        data, params = '{ ', dict()
        for k, v in node_data.items():
            pram_name = k
            pram_var_name = f'{self.variable}_{k}'
            data += f'{pram_name}: ${pram_var_name}, '
            params[pram_var_name] = v
        return data[:-2] + ' }', params
    
    def cypher(self) -> tuple[str, dict]:
        data, params = self.__build_data()
        query = (
            f'MERGE ({self.variable}:{self.__lable__} { data })\n'
            f'ON MATCH SET {self.variable}.matched = true'
        )
        return query, params
    
    @classmethod
    def __build_unique_constraint_cypher(cls, field: str) -> str:
        return (
            f'CREATE CONSTRAINT {cls.__lable__.lower()}_{field} IF NOT EXISTS\n'
            f'FOR ({cls.__variable__}:{cls.__lable__}) REQUIRE {cls.__variable__}.{field} IS UNIQUE'
        )
    
    @classmethod
    def unique_constraints_cyphers(cls) -> list[str]:
        return [cls.__build_unique_constraint_cypher(unique) for unique in cls.__unique_fields__]
    
@dataclass
class KeywordNode(CypherNode):
    __data_fields__: typing.ClassVar[list[str]] = ['text', 'type_']
    __unique_fields__: typing.ClassVar[list[str]] = ['text']
    __lable__: typing.ClassVar[str] = 'Keyword'
    __variable__: typing.ClassVar[str] = 'kw'

    type_: str
    text: str
    
    def __eq__(self, other: 'KeywordNode'):
            return self.text == other.text
        
    @property
    def identity_pattern(self) -> str:
        return f'{self.__lable__} {{text: "{self.text}"}}'
    
    @property
    def identity(self) -> str | None:
        return self.text
    
    @identity.setter
    def identity(self, value: str):
        self.text = value

    
    @classmethod
    def unique_constraints_cypher(cls) -> list[str]:
        text_constraint = (
            f'CREATE CONSTRAINT {cls.__lable__.lower()}_text IF NOT EXISTS\n'
            f'FOR ({cls.__variable__}:{cls.__lable__}) REQUIRE {cls.__variable__}.text IS UNIQUE'
        )
        return super().unique_constraints_cyphers() + [('')]
        
@dataclass
class DocNode(Doc, CypherNode, Embeded):
    __data_fields__: typing.ClassVar[list[str]] = ['id', 'title', 'text', 'link', 'embeddings']
    __unique_fields__: typing.ClassVar[list[str]] = ['id', 'title']
    __lable__: typing.ClassVar[str] = 'Doc'
    __variable__: typing.ClassVar[str] = 'doc'
    
    def __eq__(self, other: 'DocNode'):
            return self.link == other.link or self.title == other.title
        
    @property
    def identity(self) -> str | None:
        return None
    
@dataclass
class SpanNode(Span, CypherNode, Embeded):
    __data_fields__: typing.ClassVar[list[str]] = ['id', 'text', 'embeddings']
    __unique_fields__: typing.ClassVar[list[str]] = ['id']
    __lable__: typing.ClassVar[str] = 'Span'
    __variable__: typing.ClassVar[str] = 'span'
    
    def __eq__(self, other: 'SpanNode'):
            return self.id == other.id
        
    @property
    def identity(self) -> str | None:
        return None
    

@dataclass
class PlaceNode(Place, CypherNode, Embeded):
    __data_fields__: typing.ClassVar[list[str]] = [
        'id', 'address', 'category', 'city', 'lat', 'lon', 'rating', 'region',
        'region_code', 'subcategory', 'title', 'object_keys_text', 'is_modern',
        'has_music', 'fill_score', 'is_accessable', 'review_count',
        'worktime_is_24x7', 'has_wifi', 'contact_social_len', 'phone_len',
        'photos_len', 'has_website', 'biased_rating', 'embeddings']
    __unique_fields__: typing.ClassVar[list[str]] = ['id']
    __lable__: typing.ClassVar[str] = 'Place'
    __variable__: typing.ClassVar[str] = 'place'
    
    def __eq__(self, other: 'PlaceNode'):
            return self.title == other.title
        
    @property
    def identity(self) -> str | None:
        return None
    
class EntitiesManager:
    def __init__(self):
        self._last_index = 0
        
    def reset(self):
        self._last_index = 0
        
    def __gen_index(self) -> int:
        self._last_index += 1
        return self._last_index - 1
    
    def connect(self, tail: Node, head: Node, type_: str):
        relation = Relation(
            id=self.__gen_index(),
            head=head,
            tail=tail,
            type_=type_,
        )
        tail.relations.append(relation)
        head.backtracks.append(relation)
        
    def keyword(self, text: str, type_: str):
        return KeywordNode(
            id=self.__gen_index(),
            type_=type_,
            text=text,
            relations=[],
            backtracks=[],
        )
    
    def doc(self, doc: Doc):
        return DocNode(
            id=self.__gen_index(),
            title=doc.title,
            text=doc.text,
            link=doc.link,
            relations=[],
            backtracks=[],
            embeddings=[],
        )
    
    def span(self, span: str | Span):
        if issubclass(Span, span.__class__):
            span = span.text
        return SpanNode(
            id=self.__gen_index(),
            text=span,
            relations=[],
            backtracks=[],
            embeddings=[],
        )
    
    def place(self, place: Place):
        return PlaceNode(
            id=self.__gen_index(),
            relations=[],
            backtracks=[],
            embeddings=[],
            text=place.text,
            address = place.address,
            category = place.category,
            city = place.city,
            lat = place.lat,
            lon = place.lon,
            rating = place.rating,
            subcategory = place.subcategory,
            title = place.title,
            object_keys_text = place.object_keys_text,
            is_modern = place.is_modern,
            has_music = place.has_music,
            fill_score = place.fill_score,
            is_accessable = place.is_accessable,
            review_count = place.review_count,
            worktime_is_24x7 = place.worktime_is_24x7,
            has_wifi = place.has_wifi,
            contact_social_len = place.contact_social_len,
            phone_len = place.phone_len,
            photos_len = place.photos_len,
            has_website = place.has_website,
            biased_rating = place.biased_rating,
        )
    
entities_manager = EntitiesManager()

def text_glance(node_repr: str, glen: int) -> str:
    try:
        text_idx = node_repr.rfind('text')
        return node_repr[text_idx:text_idx+glen]
    except:
        return ''

def print_nodes(nodes):
    for n in nodes:
        nrepr = str(n)
        trepr = text_glance(nrepr, 50)
        print(f'{n.id} | {trepr} | {nrepr[:100]} | {len(n.relations)}')
        for r in n.relations:
            print(f'\t{r.tail.id}>{r.head.id}')
        for r in n.backtracks:
            print(f'\t{r.tail.id}<{r.head.id}')

#### test

In [23]:
def build_test_nodes():
    entities_manager.reset()

    doc_example = Doc(
        title="Example Title",
        text="This is an example text.",
        link="http://example.com"
    )
    dn = entities_manager.doc(doc_example)


    span_example = Span(
        text="Example Span"
    )
    sn = entities_manager.span(span_example)

    place_example = Place(
        text="Example Place",
        address="123 Example St",
        category="Cafe",
        city="Example City",
        lat="123.456",
        lon="-123.456",
        rating="4.5",
        subcategory="Coffee Shop",
        title="Example Cafe",
        object_keys_text="Coffee, Wifi",
        is_modern="True",
        has_music="True",
        fill_score="0.9",
        is_accessable="True",
        review_count="100",
        worktime_is_24x7="False",
        has_wifi="True",
        contact_social_len="2",
        phone_len="10",
        photos_len="5",
        has_website="True",
        biased_rating="4.2"
    )
    pn = entities_manager.place(place_example)
    
    
    kn1 = entities_manager.keyword(text="xxxx", type_='lel')
    kn2 = entities_manager.keyword(text="xxxx", type_='lel')
    kn3 = entities_manager.keyword(text="xyxx", type_='lel')
    kn4 = entities_manager.keyword(text="xyyx", type_='lel')
    
    kn5 = entities_manager.keyword(text="yyyy", type_='lel')
    
    kn6 = entities_manager.keyword(text="xtxt", type_='lel')
   
    
    entities_manager.connect(kn1, kn2, 'r')
    entities_manager.connect(kn2, kn3, 'r')
    entities_manager.connect(kn3, kn4, 'r')
    entities_manager.connect(kn4, kn5, 'r')
    entities_manager.connect(kn5, kn1, 'r')
    entities_manager.connect(kn5, kn6,'r')
    
    entities_manager.reset()
    return [kn1, kn2, kn3, kn4, kn5, kn6, dn, sn, pn]

t = build_test_nodes()
print_nodes(t)

3 | text='xxxx') | KeywordNode(id=3, relations=[Relation(id=9, head=KeywordNode(id=4, relations=[Relation(id=10, head=K | 1
	3>4
	7<3
4 | text='xxxx') | KeywordNode(id=4, relations=[Relation(id=10, head=KeywordNode(id=5, relations=[Relation(id=11, head= | 1
	4>5
	3<4
5 | text='xyxx') | KeywordNode(id=5, relations=[Relation(id=11, head=KeywordNode(id=6, relations=[Relation(id=12, head= | 1
	5>6
	4<5
6 | text='xyyx') | KeywordNode(id=6, relations=[Relation(id=12, head=KeywordNode(id=7, relations=[Relation(id=13, head= | 1
	6>7
	5<6
7 | text='yyyy') | KeywordNode(id=7, relations=[Relation(id=13, head=KeywordNode(id=3, relations=[Relation(id=9, head=K | 2
	7>3
	7>8
	6<7
8 | text='xtxt') | KeywordNode(id=8, relations=[], backtracks=[Relation(id=14, head=..., tail=KeywordNode(id=7, relatio | 0
	7<8
0 | text.', link='http://example.com') | DocNode(embeddings=[], id=0, relations=[], backtracks=[], title='Example Title', text='This is an ex | 0
1 | text='Example Span') | SpanNode(embeddings=[],

In [24]:
print(t[3].relations[0].cypher())

('MATCH (t:Keyword {text: "xyyx"}), (h:Keyword {text: "yyyy"})CREATE (t)-[:R]->(h)', {})


In [25]:
print(t[3].unique_constraints_cyphers())

['CREATE CONSTRAINT keyword_text IF NOT EXISTS\nFOR (kw:Keyword) REQUIRE kw.text IS UNIQUE']


In [26]:
# get_top_objects()

In [27]:
# TEST_MEM_ELEMENTS_AM = 100_000
# def baseline_testmem():
#     import tracemalloc
#     def app():
#         lt = []
#         for i in range(0, TEST_MEM_ELEMENTS_AM):
#             lt.append(i)
#     tracemalloc.start()
#     app()
#     print(tracemalloc.get_traced_memory())
#     tracemalloc.stop()
# testmem_baseline = baseline_testmem()
# testmem_baseline

# Pipeline

## Disambiguator

In [30]:
from pymystem3 import Mystem

In [31]:
# l=Mystem()
# ''.join(l.lemmatize('Российская академия наук')[:-1])

In [32]:
nonword_pattern = re.compile(r'[^0-9а-яА-ЯёЁa-zA-Z\s.-]+')

class BasicDisambiguator:
    def __init__(self, thesaurus: dict=dict(), lemmatizer=Mystem()):
        self.__lemmatizer = lemmatizer
        self.__thesaurus = thesaurus
        
    def disambiguate(self, node: Node) -> Node:
        if node.identity:
            node.identity = self._disambiguate(node.identity)
        return node
        
    def _disambiguate(self, identity: str) -> str:
        identity = nonword_pattern.sub(r'', identity)
        if identity in self.__thesaurus:
            return identity
        identity = ''.join(self.__lemmatizer.lemmatize(identity)[:-1])
        for core, synonims in self.__thesaurus.items():
            if identity in synonims:
                return core
        return identity
    
    def __call__(self, nodes: list[Node]) -> typing.Iterable[Node]:
        return [self.disambiguate(node) for node in nodes]
    
disambiguator = BasicDisambiguator({
    'санкт-петербург': ['петербург', 'питер'],
    'российский федерация': ['рф', 'россия'],
    'российский академия наука': ['ран']
})

#### test

In [34]:
# def test():
#     entities_manager.reset()
#     xxxn = entities_manager.keyword(text="xxx", type_='n')
#     spbn1 = entities_manager.keyword(text="Питер", type_='n')
#     spbn2 = entities_manager.keyword(text="Петербурга", type_='n')
#     ln = entities_manager.keyword(text="xxx yyy. 'zzz, 13", type_='n')
#     nodes = build_test_nodes() + [xxxn, spbn1, spbn2, ln]
#     bd = BasicDisambiguator({
#         'yyy': ['xxx'],
#         'санкт-петербург': ['питер', 'петербург'],
#         'российский федерация': ['рф', 'россия'],
#         'российский академия наука': ['ран']
#     })
#     res = bd(nodes)
#     print_nodes(nodes)
#     entities_manager.reset()
# test()

## Remover

In [36]:
class RelationsRemover:
    def remove_relations(self, node: Node) -> Node:
        """returns list of linked nodes"""
        
        # filter backtracks
        for del_rel in node.relations:
            del_rel.head.backtracks = [
                rel 
                for rel in del_rel.head.backtracks 
                if rel.tail.id != node.id
            ]
        node.relations = []

        # filter relations
        for del_rel in node.backtracks:
            del_rel.tail.relations = [
                rel 
                for rel in del_rel.tail.relations 
                if rel.head.id != node.id
            ]
        node.backtracks = []
        return node

#### test

In [38]:
# def test():
#     entities_manager.reset()
#     r = RelationsRemover()
    
#     test_nodes = build_test_nodes()
#     print_nodes(test_nodes)
#     print('\n\n')
#     _ = r.remove_relations(test_nodes[1])
#     print_nodes(test_nodes)
#     entities_manager.reset()
#     return test_nodes
# res = test()

## Filter

In [40]:
class BaseFilter(RelationsRemover):        
    def validate_one(self, node: Node) -> bool:
        return True
    
    def remove_relations(self, node):
        return super().remove_relations(node)
    
    def validate(self, nodes: list[Node]) -> typing.Iterable[Node]:
        filtered = [self.remove_relations(node) for node in nodes if not self.validate_one(node)]
        return [node for node in nodes if self.validate_one(node)]
    
    def __call__(self, nodes: list[Node]) -> typing.Iterable[Node]:
        return self.validate(nodes)

### StopwordsFilter

In [42]:
class StopwordsFilter(BaseFilter):
    def __init__(self, stop_sub_strings: list[str]):
        self.__stop_sub_strings = stop_sub_strings
        
    def validate_one(self, node: Node) -> bool:
        if node.identity:
            return node.identity not in self.__stop_sub_strings
        return True
    
stopwords_filter = StopwordsFilter([
    'пресс-служба', 'ресурс', 'политика',
    'пао', 'хорошо', 'Kion', 'правильно',
    'два', 'оао',
])

#### test

In [44]:
# def test():
#     entities_manager.reset()
#     print('xyxx')
#     sw_filter = StopwordsFilter(['xyxx'])
#     test_nodes = build_test_nodes()
#     print_nodes(test_nodes); print('\n\n')
#     res = sw_filter(test_nodes)
#     print_nodes(res)
#     entities_manager.reset()
#     return res
# _ = test()

### SizeFilter

In [46]:
class SizeFilter(BaseFilter):
    def __init__(self, threshold: int):
        self.__threshold = threshold
        
    def validate_one(self, node: Node) -> bool:
        if node.identity:
            return len(node.identity) > self.__threshold
        return True
                    
size_filter = SizeFilter(1)

In [47]:
# def test_mem():
#     import tracemalloc
#     sss = StopSubStringsValidator(['xxx'])
#     vals = [KeywordNode(i, [1, 2], text=f"xxxx{i}") for i in range(10_000)]  
    
#     tracemalloc.start()
    
#     for v in vals:
#         sss.validate(v)

#     print(tracemalloc.get_traced_memory())
#     tracemalloc.stop()
#     print(gc.collect())
# test_mem()

## Comparator

In [49]:
class BaseIdentityComparator(ABC):
    def equals(self, l: Node, r: Node) -> bool:
        if l.identity and r.identity:
            return self._equals(l.identity, r.identity)
        return False

class EqualsComparator(BaseIdentityComparator):
    def _equals(self, l: str, r: str) -> bool:
        return l == r

class HammingComparator(BaseIdentityComparator):
    def __init__(self, thershold: int=3, size_thershold=6):
        self.__thershold = thershold
        self.__size_thershold = size_thershold
        
    def _equals(self, l: str, r: str) -> bool:
        min_len = min(len(l), len(r))
        if min_len <= self.__thershold or min_len < self.__size_thershold:
            return l == r
        return sum(l_char != r_char for l_char, r_char in zip(l, r)) <= self.__thershold

#### test

In [51]:
# def text(hc, ec):
#     entities_manager.reset()
#     xxxxxxx1 = entities_manager.keyword(text="xxxxxxx", type_='lel')
#     yyyyyyy = entities_manager.keyword(text="yyyyyyy", type_='lel')
#     xxxxxxx2 = entities_manager.keyword(text="xxxxxxx", type_='lel')
#     xyxxyxx = entities_manager.keyword(text="xyxxyxx", type_='lel') 

#     sports = entities_manager.keyword(text="sports", type_='lel')
#     food = entities_manager.keyword(text="food", type_='lel')
    
#     entities_manager.reset()
#     return (
#         ec.equals(xxxxxxx1, yyyyyyy),
#         hc.equals(xxxxxxx1, yyyyyyy),
        
#         ec.equals(xxxxxxx1, xxxxxxx2),
#         hc.equals(xxxxxxx1, xxxxxxx2),        

#         ec.equals(xxxxxxx1, xyxxyxx),
#         hc.equals(xxxxxxx1, xyxxyxx),

#         ec.equals(sports, food),
#         hc.equals(sports, food),
#     )

# text(HammingComparator(), EqualsComparator())

## Deduplicator

In [53]:
class Comparator(typing.Protocol):
    def equals(l: Node, r: Node) -> bool:
        pass
    
class BasicDeduplicator:
    def __init__(self, comparators: list[Comparator]):
        self.__comparators = comparators
        
    def check_equals(self, l: Node, r: Node) -> bool:
        for c in self.__comparators:
            if c.equals(l, r):
                return True
        return False
    
    def __move_relations(self, target: Node, source: Node):
        for rel in source.relations:
            rel.tail = target
            target.relations.append(rel)
            
        for rel in source.backtracks:
            rel.head = target
            target.backtracks.append(rel)
            
        return target
    
    def __deduplicate_nodes(self, nodes: list[Node]) -> list[Node]:
        nodes_dict = {n.id: n for n in nodes}
        united_nodes_ids = set()
        merged_nodes_ids = set()
        
        for l_node in nodes:
            if l_node.id in united_nodes_ids or l_node.id in merged_nodes_ids:
                continue
            united_nodes_ids.add(l_node.id)
            
            for r_node in nodes:                
                if r_node.id in united_nodes_ids or r_node.id in merged_nodes_ids:
                    continue
                    
                if self.check_equals(l_node, r_node):
                    l_node = self.__move_relations(l_node, r_node)
                    merged_nodes_ids.add(r_node.id)
                    del nodes_dict[r_node.id]
                    continue
                    
        return list(nodes_dict.values())
    
    def __deduplicate_and_decycle_links(self, node: Node) -> Node:
        unique_relations = {rel.head.id: rel for rel in node.relations}
        if node.id in unique_relations:
            del unique_relations[node.id ]
        node.relations = list(unique_relations.values())
        
        unique_backtracks = {rel.tail.id: rel for rel in node.backtracks}
        if node.id in unique_backtracks:
            del unique_backtracks[node.id ]
        node.backtracks = list(unique_backtracks.values())
        return node

    
    def deduplicate(self, nodes: list[Node]) -> list[Node]:
        nodes = self.__deduplicate_nodes(nodes)
        return [self.__deduplicate_and_decycle_links(node) for node in nodes]
    
    def __call__(self, nodes: typing.Iterable[Node]) -> typing.Iterable[Node]:
        return self.deduplicate(nodes)

    
equals_comparator = EqualsComparator()
hamming_comparator = HammingComparator()
deduplicator = BasicDeduplicator([equals_comparator, hamming_comparator])

#### test

In [55]:
# def test(deduplicator=None):
#     entities_manager.reset()
#     bc = EqualsComparator()
#     hc = HammingComparator(size_thershold=3)
#     d = deduplicator if deduplicator else BasicDeduplicator([bc, hc])
    
#     test_nodes = build_test_nodes()
#     print_nodes(test_nodes)
#     print('\n\n')
#     res = d(test_nodes)
#     print_nodes(res)
#     entities_manager.reset()
#     return res
# res = test(deduplicator)

Gold:
```
0 | text='xxxx') | KeywordNode(id_=0, relations=[Relation(id_=6, head=KeywordNode(id_=1, relations=[Relation(id_=7, hea | 1
	0>2
	4<0
2 | text='xyxx') | KeywordNode(id_=2, relations=[Relation(id_=8, head=KeywordNode(id_=3, relations=[Relation(id_=9, hea | 1
	2>3
	0<2
3 | text='xyyx') | KeywordNode(id_=3, relations=[Relation(id_=9, head=KeywordNode(id_=4, relations=[Relation(id_=10, he | 1
	3>4
	2<3
4 | text='yyyy') | KeywordNode(id_=4, relations=[Relation(id_=10, head=KeywordNode(id_=0, relations=[Relation(id_=6, he | 2
	4>0
	4>5
	3<4
5 | text='xtxt') | KeywordNode(id_=5, relations=[], backtracks=[Relation(id_=11, head=..., tail=KeywordNode(id_=4, rela | 0
	4<5
```

In [57]:
# def test_mem():
#     bc = BasicComparator()
#     hc = HammingComparator()
#     d = BasicDeduplicator([bc, hc])
#     import tracemalloc
#     vals = [KeywordNode(i, [1, 2], text=f"xxxx{i}") for i in range(TEST_MEM_ELEMENTS_AM)]
#     tracemalloc.start()
#     d.deduplicate(vals)
#     print(tracemalloc.get_traced_memory())
#     tracemalloc.stop()
#     print(gc.collect())
# testmem_ded = test_mem()

In [58]:
# (2617, 3995369)
# 12818350 / 3995369

## Splitter

In [60]:
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 300

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    length_function=len,
    is_separator_regex=False,
)

## NER&RE

### Spacy

In [63]:
nlp = spacy.load('ru_core_news_sm')

In [64]:
class SpacyExtractor:
    def __init__(self, entities_manager, nlp=None):
        self._entities_manager = entities_manager
        self._nlp = nlp if nlp else spacy.load('ru_core_news_sm')
        self._doc = None

    def extract(self, text: str) -> typing.Iterable[Node]:
        text = text.replace('\n', ' ')
        self._doc = self._nlp(text)
        return [self._node_from_span(span) for span in self._doc.ents]
    
    def _node_from_span(self, ent: spacy.tokens.span.Span) -> Node:
        return self._entities_manager.keyword(text=ent.lemma_, type_=ent.label_)

    def __call__(self, text: str) -> typing.Iterable[Node]:
        return self.extract(text)
    
spacy_extractor = SpacyExtractor(entities_manager, nlp)

#### test

In [66]:
# text = 'Колоннада Исаакиевского собора: Вид с высоты на главные достопримечательности Петербурга\nКолоннада Исаакиевского собора: Вид с высоты на главные достопримечательности Петербурга\n Войти Санкт-Петербург и Ленинградская область\nКолоннада Исаакиевского собора\nПутеводители\nКолоннада Исаакиевского собора\nВид с высоты на главные достопримечательности Петербурга\n \nЗачем забираться под купол Исаакиевского собора? 262 ступени по винтовой лестнице! Не проще ли на лифте подняться в бар в зданиях по соседству и оттуда бросить взгляд на колоннаду, Адмиралтейство и остальные достопримечательности? Проще. Но посещение колоннады — приключение совсем другого толка. Смотровая площадка с колоннами открыта всем ветрам, лучам, дождю и снегу. Оттуда можно рассмотреть крышу, купол, шпиль собора, а ещё морской порт, Адмиралтейство и стадион «Газпром-арена» как на ладони.\nИсаакиевская площадь, 4\nЮлия Тарнавская\nпродюсер\nРядом с Исаакием, на Большой Морской, 45, есть неплохой бар — WöD. А при нём ещё и детская площадка. Кстати, домом этим недолгое время владел архитектор Огюст Монферран, который 40 лет строил Исаакиевский собор.\nНазад к путеводителю\n \xa0\n- бесплатно на территории РФ\n \xa0\n- для абонентов – бесплатно при звонке из\n \nцелого ряда стран\n,\n - Пресс-служба - Редакция - По общим вопросам\nРесурсы\nКомплаенс и деловая этика\nПриложение «Мой\xa0 »\nДокументы \nДокументы ПАО\xa0 \nСайт \nПравила получения промокодов\nПолитика В приложении цены на отели ещё ниже, а выбор шире!\n Ваучер хранится в личном кабинете и доступен, даже если нет интернета.\n© 2023 ПАО « ». , \nОставаясь с нами, вы соглашаетесь\n на использование \nфайлов куки\nХОРОШО'

In [67]:
# def test_spacy_extractor():
#     entities_manager.reset()
#     print_nodes(spacy_extractor(text))
# test_spacy_extractor()

In [68]:
# get_top_objects()

### Wikineural

In [70]:
tokenizer = AutoTokenizer.from_pretrained("Babelscape/wikineural-multilingual-ner")
model = AutoModelForTokenClassification.from_pretrained("Babelscape/wikineural-multilingual-ner")
nlp = pipeline("ner", model=model, tokenizer=tokenizer,aggregation_strategy=AggregationStrategy.SIMPLE)

In [71]:
class WikineuralExtractor:
    def __init__(self, entities_manager, text_splitter, nlp=None, threshold=0.6):        
        if not nlp:
            tokenizer = AutoTokenizer.from_pretrained("Babelscape/wikineural-multilingual-ner")
            model = AutoModelForTokenClassification.from_pretrained("Babelscape/wikineural-multilingual-ner")
            nlp = pipeline("ner", model=model, tokenizer=tokenizer,aggregation_strategy=AggregationStrategy.SIMPLE)
        
        self.__text_splitter = text_splitter
        self._entities_manager = entities_manager
        self._nlp = nlp
        self._threshold = threshold
        self._text = None
        
    def extract(self, text: str) -> typing.Iterable[Node]:
        text_spans = self.__text_splitter.create_documents([text])
        return [node for span in text_spans for node in self._extract(span.page_content)]
        
    def _extract(self, text: str) -> typing.Iterable[Node]:
        text = text.replace('\n', ' ')
        self._text = text
        spans = self._nlp(text)
        return [self._node_from_span(span) for span in spans if span['score'] > self._threshold]
    
    def _node_from_span(self, span: dict) -> Node:
        return self._entities_manager.keyword(
            text=self._text[span['start']:span['end']], type_=span['entity_group'])

    def __call__(self, text: str) -> typing.Iterable[Node]:
        return self.extract(text)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=0,
    length_function=len,
    is_separator_regex=False,
)

wikineural_extractor = WikineuralExtractor(entities_manager, text_splitter, nlp=nlp)

#### test

In [73]:
# text = 'Колоннада Исаакиевского собора: Вид с высоты на главные достопримечательности Петербурга\nКолоннада Исаакиевского собора: Вид с высоты на главные достопримечательности Петербурга\n Войти Санкт-Петербург и Ленинградская область\nКолоннада Исаакиевского собора\nПутеводители\nКолоннада Исаакиевского собора\nВид с высоты на главные достопримечательности Петербурга\n \nЗачем забираться под купол Исаакиевского собора? 262 ступени по винтовой лестнице! Не проще ли на лифте подняться в бар в зданиях по соседству и оттуда бросить взгляд на колоннаду, Адмиралтейство и остальные достопримечательности? Проще. Но посещение колоннады — приключение совсем другого толка. Смотровая площадка с колоннами открыта всем ветрам, лучам, дождю и снегу. Оттуда можно рассмотреть крышу, купол, шпиль собора, а ещё морской порт, Адмиралтейство и стадион «Газпром-арена» как на ладони.\nИсаакиевская площадь, 4\nЮлия Тарнавская\nпродюсер\nРядом с Исаакием, на Большой Морской, 45, есть неплохой бар — WöD. А при нём ещё и детская площадка. Кстати, домом этим недолгое время владел архитектор Огюст Монферран, который 40 лет строил Исаакиевский собор.\nНазад к путеводителю\n \xa0\n- бесплатно на территории РФ\n \xa0\n- для абонентов – бесплатно при звонке из\n \nцелого ряда стран\n,\n - Пресс-служба - Редакция - По общим вопросам\nРесурсы\nКомплаенс и деловая этика\nПриложение «Мой\xa0 »\nДокументы \nДокументы ПАО\xa0 \nСайт \nПравила получения промокодов\nПолитика В приложении цены на отели ещё ниже, а выбор шире!\n Ваучер хранится в личном кабинете и доступен, даже если нет интернета.\n© 2023 ПАО « ». , \nОставаясь с нами, вы соглашаетесь\n на использование \nфайлов куки\nХОРОШО'

In [74]:
# text_splitter.create_documents([text])

In [75]:
# wikineural_extractor(text)

In [76]:
# def test_wikineural_extractor():
#     entities_manager.reset()
#     print_nodes(wikineural_extractor(text))
# test_wikineural_extractor()

### Natasha

In [78]:
# class NatashaExtractor:
#     def __init__(self, entities_manager):
#         self._entities_manager = entities_manager
#         self._segmenter = natasha.Segmenter()
#         self._morph_vocab = natasha.MorphVocab()
#         self._names_extractor = natasha.NamesExtractor(self._morph_vocab)
#         self._embeddings = natasha.NewsEmbedding()
#         self._morph_tagger = natasha.NewsMorphTagger(self._embeddings)
#         self._ner_tagger = natasha.NewsNERTagger(self._embeddings)
#         self._doc = None

#     def extract(self, text: str) -> typing.Iterable[Node]:
#         text = text.replace('\n', ' ')
#         self._doc = natasha.Doc(text)
#         self._doc.segment(self._segmenter)
#         self._doc.tag_ner(self._ner_tagger)
#         return [self._node_from_span(span) 
#                      for span in self._doc.ner.spans]
    
#     def _node_from_span(self, span: natasha.span.Span) -> Node:
#         return self._entities_manager.keyword(
#             text=self._doc.text[span.start:span.stop], type_=span.type)

#     def __call__(self, text: str) -> typing.Iterable[Node]:
#         return self.extract(text)
    
# natasha_extractor = NatashaExtractor(entities_manager)

#### test

In [80]:
# get_top_objects()

In [81]:
# ttxt = 'Колоннада Исаакиевского собора: Вид с высоты на главные достопримечательности Петербурга\nКолоннада Исаакиевского собора: Вид с высоты на главные достопримечательности Петербурга\n Войти Санкт-Петербург и Ленинградская область\nКолоннада Исаакиевского собора\nПутеводители\nКолоннада Исаакиевского собора\nВид с высоты на главные достопримечательности Петербурга\n \nЗачем забираться под купол Исаакиевского собора? 262 ступени по винтовой лестнице! Не проще ли на лифте подняться в бар в зданиях по соседству и оттуда бросить взгляд на колоннаду, Адмиралтейство и остальные достопримечательности? Проще. Но посещение колоннады — приключение совсем другого толка. Смотровая площадка с колоннами открыта всем ветрам, лучам, дождю и снегу. Оттуда можно рассмотреть крышу, купол, шпиль собора, а ещё морской порт, Адмиралтейство и стадион «Газпром-арена» как на ладони.\nИсаакиевская площадь, 4\nЮлия Тарнавская\nпродюсер\nРядом с Исаакием, на Большой Морской, 45, есть неплохой бар — WöD. А при нём ещё и детская площадка. Кстати, домом этим недолгое время владел архитектор Огюст Монферран, который 40 лет строил Исаакиевский собор.\nНазад к путеводителю\n \xa0\n- бесплатно на территории РФ\n \xa0\n- для абонентов – бесплатно при звонке из\n \nцелого ряда стран\n,\n - Пресс-служба - Редакция - По общим вопросам\nРесурсы\nКомплаенс и деловая этика\nПриложение «Мой\xa0 »\nДокументы \nДокументы ПАО\xa0 \nСайт \nПравила получения промокодов\nПолитика В приложении цены на отели ещё ниже, а выбор шире!\n Ваучер хранится в личном кабинете и доступен, даже если нет интернета.\n© 2023 ПАО « ». , \nОставаясь с нами, вы соглашаетесь\n на использование \nфайлов куки\nХОРОШО'
# def test_natasha_extractor():
#     entities_manager.reset()
#     print_nodes(natasha_extractor(ttxt))
# test_natasha_extractor()

### Mrebel

In [83]:
bmodel = transformers.AutoModelForSeq2SeqLM.from_pretrained(
            "Babelscape/mrebel-base")
btokenizer = transformers.AutoTokenizer.from_pretrained(
            "Babelscape/mrebel-base", src_lang="ru", tgt_lang="ru")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [84]:
import torch; torch.backends.mps.is_available()
DEVICE = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
bmodel.to(DEVICE)

M2M100ForConditionalGeneration(
  (model): M2M100Model(
    (shared): Embedding(128129, 1024, padding_idx=1)
    (encoder): M2M100Encoder(
      (embed_tokens): Embedding(128129, 1024, padding_idx=1)
      (embed_positions): M2M100SinusoidalPositionalEmbedding()
      (layers): ModuleList(
        (0-11): 12 x M2M100EncoderLayer(
          (self_attn): M2M100Attention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNorm

In [85]:
class MRebellExtractor:
    def __init__(self,
        entities_manager,
        tokenizer = None,
        model = None,
        gen_kwargs: dict = None,   
        ):
        self._entities_manager = entities_manager
        if not tokenizer:
            tokenizer = transformers.AutoTokenizer.from_pretrained(
                "Babelscape/mrebel-base", src_lang="ru", tgt_lang="ru")
        self._tokenizer = tokenizer
        if not model:
            model = transformers.AutoModelForSeq2SeqLM.from_pretrained(
                        "Babelscape/mrebel-base")
        self._model = model
        if not gen_kwargs:
            gen_kwargs = {
                "max_length": 512, #256,
                "length_penalty": 0,
                "num_beams": 3,
                "num_return_sequences": 3,
                "forced_bos_token_id": None,
            }
        self._gen_kwargs = gen_kwargs

    def extract(self, text: str) -> typing.Iterable[Node]:
        logger.debug('extracting with MRebellExtractor...')
        model_inputs = self._tokenizer(
            text, 
            max_length=256, 
            padding=True, 
            truncation=True, 
            return_tensors = 'pt',
        )
        logger.debug('MRebellExtractor tokenization done.')
        generated_tokens = self._model.generate(
            model_inputs["input_ids"].to(self._model.device),
            attention_mask=model_inputs["attention_mask"].to(self._model.device),
            **self._gen_kwargs,
        )
        logger.debug('MRebellExtractor model work done.')
        preds_packs = self._tokenizer.batch_decode(
            generated_tokens, skip_special_tokens=False)
        logger.debug('MRebellExtractor decode done.')
        triplets = self._triplets(preds_packs)
        logger.debug('MRebellExtractor triplets decode done.')
        return self._nodes_from_triplets(triplets)

    def _triplets(self, preds_packs):
        triplets = []
        for pack in preds_packs:
            triplets += self._triplets_from_preds(pack)
        return triplets
            
    def _triplets_from_preds(self, preds):
        triplets = []
        relation = ''
        text = preds.strip()
        current = 'x'
        subject, relation, object_, object_type, subject_type = '','','','',''

        for token in text.replace("<s>", "").replace("<pad>", "").replace("</s>", "").replace("tp_XX", "").replace("__ru__", "").split():
            if token == "<triplet>" or token == "<relation>":
                current = 't'
                if relation != '':
                    triplets.append({'head': subject.strip(), 'head_type': subject_type, 'type': relation.strip(),'tail': object_.strip(), 'tail_type': object_type})
                    relation = ''
                subject = ''
            elif token.startswith("<") and token.endswith(">"):
                if current == 't' or current == 'o':
                    current = 's'
                    if relation != '':
                        triplets.append({'head': subject.strip(), 'head_type': subject_type, 'type': relation.strip(),'tail': object_.strip(), 'tail_type': object_type})
                    object_ = ''
                    subject_type = token[1:-1]
                else:
                    current = 'o'
                    object_type = token[1:-1]
                    relation = ''
            else:
                if current == 't':
                    subject += ' ' + token
                elif current == 's':
                    object_ += ' ' + token
                elif current == 'o':
                    relation += ' ' + token
        if subject != '' and relation != '' and object_ != '' and object_type != '' and subject_type != '':
            triplets.append({'head': subject.strip(), 'head_type': subject_type, 'type': relation.strip(),'tail': object_.strip(), 'tail_type': object_type})
        return triplets

    def _keywords_from_text(self, text) -> list[str]:
        return [kw.strip() for kw in text.split(',')]
    
    def _relation_type_from_triplet(self, triplet: dict):
        return triplet['type']

    def _nodes_from_triplet_part(self, triplet: dict, part: str) -> list[Node]:
        node_text = triplet[part]
        node_type = triplet[f'{part}_type']
        part_nodes = list()
        raw_node_keywords = self._keywords_from_text(node_text)
        for kw in raw_node_keywords:
            part_nodes.append(self._entities_manager.keyword(
                text=kw, 
                type_=node_type, 
            ))
        return part_nodes

    def _nodes_from_triplet(self, triplet: dict) -> list[Node]:
        head_nodes = self._nodes_from_triplet_part(triplet, 'head') # у этой модели инверсия tail = head
        tail_nodes = self._nodes_from_triplet_part(triplet, 'tail') # у этой модели инверсия head = tail
        relation_type = self._relation_type_from_triplet(triplet)
        for tail_node in tail_nodes:
            for head_node in head_nodes:
                self._entities_manager.connect(
                    tail=tail_node,
                    head=head_node,
                    type_=relation_type,
                )
        return head_nodes + tail_nodes

    def _nodes_from_triplets(self, triplets: list[dict]) -> typing.Iterable[Node]:
        return [node for triplet in triplets for node in self._nodes_from_triplet(triplet)]
    
    def __call__(self, text: str) -> typing.Iterable[Node]:
        return self.extract(text)

    
mrebell_extractor = MRebellExtractor(entities_manager, model=bmodel, tokenizer=btokenizer)

#### test

In [87]:
# ttxt = 'Колоннада Исаакиевского собора: Вид с высоты на главные достопримечательности Петербурга\nКолоннада Исаакиевского собора: Вид с высоты на главные достопримечательности Петербурга\n Войти Санкт-Петербург и Ленинградская область\nКолоннада Исаакиевского собора\nПутеводители\nКолоннада Исаакиевского собора\nВид с высоты на главные достопримечательности Петербурга\n \nЗачем забираться под купол Исаакиевского собора? 262 ступени по винтовой лестнице! Не проще ли на лифте подняться в бар в зданиях по соседству и оттуда бросить взгляд на колоннаду, Адмиралтейство и остальные достопримечательности? Проще. Но посещение колоннады — приключение совсем другого толка. Смотровая площадка с колоннами открыта всем ветрам, лучам, дождю и снегу. Оттуда можно рассмотреть крышу, купол, шпиль собора, а ещё морской порт, Адмиралтейство и стадион «Газпром-арена» как на ладони.\nИсаакиевская площадь, 4\nЮлия Тарнавская\nпродюсер\nРядом с Исаакием, на Большой Морской, 45, есть неплохой бар — WöD. А при нём ещё и детская площадка. Кстати, домом этим недолгое время владел архитектор Огюст Монферран, который 40 лет строил Исаакиевский собор.\nНазад к путеводителю\n \xa0\n- бесплатно на территории РФ\n \xa0\n- для абонентов – бесплатно при звонке из\n \nцелого ряда стран\n,\n - Пресс-служба - Редакция - По общим вопросам\nРесурсы\nКомплаенс и деловая этика\nПриложение «Мой\xa0 »\nДокументы \nДокументы ПАО\xa0 \nСайт \nПравила получения промокодов\nПолитика В приложении цены на отели ещё ниже, а выбор шире!\n Ваучер хранится в личном кабинете и доступен, даже если нет интернета.\n© 2023 ПАО « ». , \nОставаясь с нами, вы соглашаетесь\n на использование \nфайлов куки\nХОРОШО'
# def test_mrebell_extractor():
#     entities_manager.reset()
#     print_nodes(mrebell_extractor(ttxt))
# test_mrebell_extractor()

## Embeder

+ https://medium.com/@therobbrennan/explore-openai-vector-embedding-with-neo4j-6ea2a40693d9
+ https://blog.langchain.dev/implementing-advanced-retrieval-rag-strategies-with-neo4j/
+ https://neo4j.com/docs/cypher-manual/current/indexes/semantic-indexes/vector-indexes/
+ https://medium.com/neo4j/building-an-educational-chatbot-for-graphacademy-with-neo4j-f707c4ce311b
+ https://riino.site/blog/2023-11-26-neo4j_llm

In [90]:
from sentence_transformers import SentenceTransformer

import transformers
import torch

import numpy as np
from numpy.linalg import norm

In [91]:
def cosine(A, B):
    return np.dot(A,B)/(norm(A)*norm(B))

In [92]:
class BaseTextEmbedder:
    def embed(self, text: str) -> list[float]:
        return []
    
    def __call__(self, text: str) -> list[float]:
        return self.embed(text)

### RubertTinyEmbedder

In [94]:
# rttokenizer = transformers.AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")
# rtmodel = transformers.AutoModel.from_pretrained("cointegrated/rubert-tiny")

# # import torch; torch.backends.mps.is_available()
# # DEVICE = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
# # model.to(DEVICE)

In [95]:
# class RubertTinyEmbedder(BaseTextEmbedder):
#     def __init__(self,
#         tokenizer = None,
#         model = None,
#         ):
#         if not tokenizer:
#             tokenizer = transformers.AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")
#         self._tokenizer = tokenizer
#         if not model:
#             model = transformers.AutoModel.from_pretrained("cointegrated/rubert-tiny")
#         self._model = model
        
    
#     def _embeddings_from_text(self, text: str) -> list[float]:
#         t = self._tokenizer(text, padding=True, truncation=True, return_tensors='pt')
        
#         with torch.no_grad():
#             model_output = self._model(**{k: v.to(self._model.device) for k, v in t.items()})
            
#         embeddings = model_output.last_hidden_state[:, 0, :]
#         embeddings = torch.nn.functional.normalize(embeddings)
#         return embeddings[0].cpu().numpy().tolist()
    
#     def embed(self, text: str) -> list[float]:
#         return self._embeddings_from_text(text)
    
# rubert_tiny_embedder = RubertTinyEmbedder(rttokenizer, rtmodel)

#### test

In [97]:
# text = 'Колоннада Исаакиевского собора: Вид с высоты на главные достопримечательности Петербурга\nКолоннада Исаакиевского собора: Вид с высоты на главные достопримечательности Петербурга\n Войти Санкт-Петербург и Ленинградская область\nКолоннада Исаакиевского собора\nПутеводители\nКолоннада Исаакиевского собора\nВид с высоты на главные достопримечательности Петербурга\n \nЗачем забираться под купол Исаакиевского собора? 262 ступени по винтовой лестнице! Не проще ли на лифте подняться в бар в зданиях по соседству и оттуда бросить взгляд на колоннаду, Адмиралтейство и остальные достопримечательности? Проще. Но посещение колоннады — приключение совсем другого толка. Смотровая площадка с колоннами открыта всем ветрам, лучам, дождю и снегу. Оттуда можно рассмотреть крышу, купол, шпиль собора, а ещё морской порт, Адмиралтейство и стадион «Газпром-арена» как на ладони.\nИсаакиевская площадь, 4\nЮлия Тарнавская\nпродюсер\nРядом с Исаакием, на Большой Морской, 45, есть неплохой бар — WöD. А при нём ещё и детская п'

In [98]:
# t = rubert_tiny_embedder(text)
# t[:10], len(t)

### MiniLM

In [100]:
mlmodel = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

In [101]:
class MiniLMEmbedder(BaseTextEmbedder):
    def __init__(self, model = None):
        self._model = model if model else SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

    def embed(self, text: str) -> list[float]:
        return self._model.encode(text).tolist()

In [102]:
miniLM_embedder = MiniLMEmbedder(mlmodel)

#### test

In [104]:
# text = 'Колоннада Исаакиевского собора: Вид с высоты на главные достопримечательности Петербурга\nКолоннада Исаакиевского собора: Вид с высоты на главные достопримечательности Петербурга\n Войти Санкт-Петербург и Ленинградская область\nКолоннада Исаакиевского собора\nПутеводители\nКолоннада Исаакиевского собора\nВид с высоты на главные достопримечательности Петербурга\n \nЗачем забираться под купол Исаакиевского собора? 262 ступени по винтовой лестнице! Не проще ли на лифте подняться в бар в зданиях по соседству и оттуда бросить взгляд на колоннаду, Адмиралтейство и остальные достопримечательности? Проще. Но посещение колоннады — приключение совсем другого толка. Смотровая площадка с колоннами открыта всем ветрам, лучам, дождю и снегу. Оттуда можно рассмотреть крышу, купол, шпиль собора, а ещё морской порт, Адмиралтейство и стадион «Газпром-арена» как на ладони.\nИсаакиевская площадь, 4\nЮлия Тарнавская\nпродюсер\nРядом с Исаакием, на Большой Морской, 45, есть неплохой бар — WöD. А при нём ещё и детская п'

In [105]:
# t = miniLM_embedder(text)
# t[:10], len(t)

In [106]:
# title = 'Юсуповский дворец — дом богатейшей семьи Российской империи и место убийства Григория Распутина'
# test_embeds = miniLM_embedder.embed(title)
# test_embeds

### NodesEmbedder

In [108]:
class TextEmbedder(typing.Protocol):  
    def embed(self, text: str) -> list[float]:
        pass
    
    def __call__(self, text: str) -> list[float]:
        pass

In [109]:
class NodesEmbedder:
    def __init__(self, text_embedder):
        self.text_embedder = text_embedder
        
    def embed_node(self, node: Node) -> Node:
        node.embeddings = self.text_embedder(node.text)
        return node
    
    def __call__(self, nodes: typing.Iterable[Node]) -> typing.Iterable[Node]:
        return [self.embed_node(node) for node in nodes]
    
    
class DocsEmbedder(NodesEmbedder):
    def embed_node(self, node: Node) -> Node:
        if not issubclass(DocNode, node.__class__):
            return node
        node.embeddings = self.text_embedder(node.title)
        return node
    
class SpansEmbedder(NodesEmbedder):
    def embed_node(self, node: Node) -> Node:
        if not issubclass(SpanNode, node.__class__):
            return node
        node.embeddings = self.text_embedder(node.text)
        return node
    
class PlacesEmbedder(NodesEmbedder):
    def embed_node(self, node: Node) -> Node:
        if not issubclass(PlaceNode, node.__class__):
            return node
        node.embeddings = self.text_embedder(node.object_keys_text)
        return node

embedders = [
    DocsEmbedder(miniLM_embedder),
    SpansEmbedder(miniLM_embedder),
    PlacesEmbedder(miniLM_embedder),
]

#### test

In [111]:
# def test():    
#     entities_manager.reset()
#     test_nodes = build_test_nodes()
#     print_nodes(test_nodes); print('\n\n---\n\n')
#     for e in embedders:
#         test_nodes = e(test_nodes)
#     print_nodes(test_nodes)
#     entities_manager.reset()
#     return test_nodes
# res = test()

# Connection

const:
```
CREATE CONSTRAINT keyword_text IF NOT EXISTS
FOR (kw:Keyword) REQUIRE kw.text IS UNIQUE
```

del:
```
MATCH (n)
OPTIONAL MATCH (n)-[r]-()
WITH n,r LIMIT 50000
DELETE n,r
RETURN count(n) as deletedNodesCount
```

inx:
```
CREATE VECTOR INDEX `doc_embeddings` IF NOT EXISTS
FOR (n: Doc) ON (n.embeddings)
OPTIONS {indexConfig: {
 `vector.dimensions`: 312,
 `vector.similarity_function`: 'cosine'
}}

CREATE VECTOR INDEX `span_embeddings` IF NOT EXISTS
FOR (n: Span) ON (n.embeddings)
OPTIONS {indexConfig: {
 `vector.dimensions`: 312,
 `vector.similarity_function`: 'cosine'
}}

CREATE VECTOR INDEX `place_embeddings` IF NOT EXISTS
FOR (n: Place) ON (n.embeddings)
OPTIONS {indexConfig: {
 `vector.dimensions`: 312,
 `vector.similarity_function`: 'cosine'
}}

```

In [114]:
from neo4j import GraphDatabase

class Neo4jConnection:
    def __init__(self, uri=None, user=None, password=None):
        self.__uri = "bolt://localhost:7687" if uri is None else uri
        self.__user = "neo4j" if user is None else user
        self.__password = "qwertyqwerty" if password is None else password
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__password))
        except Exception as e:
            logger.error("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            logger.debug()
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            logger.error(f'Query failed: {query}, {parameters}', e)
        finally: 
            if session is not None:
                session.close()
        return response
    
conn = Neo4jConnection(
    uri='bolt://46.148.235.218:7687',
    # user='neo4j',
    # password='N-ubQ5xs0y3aA-eAiM0yWSCDEupCLTaMmLFgoLT9x5g',
)
# conn.query('match (n) return n')

In [115]:
conn.query('''MATCH (n)
OPTIONAL MATCH (n)-[r]-()
WITH n,r LIMIT 50000
DELETE n,r
RETURN count(n) as deletedNodesCount''')

MATCH (n)
OPTIONAL MATCH (n)-[r]-()
WITH n,r LIMIT 50000
DELETE n,r
RETURN count(n) as deletedNodesCount, None


[<Record deletedNodesCount=0>]

In [116]:
for cq in (
    KeywordNode.unique_constraints_cyphers() 
    + DocNode.unique_constraints_cyphers()
    + SpanNode.unique_constraints_cyphers()
    + PlaceNode.unique_constraints_cyphers()
):
    print(conn.query(cq))

CREATE CONSTRAINT keyword_text IF NOT EXISTS
FOR (kw:Keyword) REQUIRE kw.text IS UNIQUE, None
[]
CREATE CONSTRAINT doc_id IF NOT EXISTS
FOR (doc:Doc) REQUIRE doc.id IS UNIQUE, None
[]
CREATE CONSTRAINT doc_title IF NOT EXISTS
FOR (doc:Doc) REQUIRE doc.title IS UNIQUE, None
[]
CREATE CONSTRAINT span_id IF NOT EXISTS
FOR (span:Span) REQUIRE span.id IS UNIQUE, None
[]
CREATE CONSTRAINT place_id IF NOT EXISTS
FOR (place:Place) REQUIRE place.id IS UNIQUE, None
[]


# Graph

## base

In [119]:
class RelationType(Enum):
    CONTAINS = 'CONTAINS'
    PART_OF_TEXT = 'PART_OF_TEXT'
    LOCATED_IN_THE_ATE = 'LOCATED_IN_THE_ADMINISTRATIVE_TERRITORIAL_ENTITY'
    HAS_CATEGORY = 'HAS_CATEGORY'

In [120]:
def print_cypher(query: str):
    for line in query.split('\n'):
        logger.debug(f'{line}')

In [121]:
class Cypherable(typing.Protocol):
    def cypher(self) -> tuple[str, dict]:
        pass
    
class Processor(typing.Protocol):
    def __call__(nodes: typing.Iterable[Node]) -> typing.Iterable[Node]:
        pass

class BaseGrapher:
    def __init__(self, 
                 conn, 
                 batch_size: int=1000,
            ):
        self._conn = conn
        self._batch_size = batch_size

    def _make_batches(self, items: typing.Iterable[Cypherable], size):
        for i in range(0, len(items), size):
            yield items[i:i + size]
    
    def _nodes_cypher(self, nodes: typing.Iterable[Node]) -> tuple[str, dict]:
        entities = sorted(nodes, key=lambda n: n.priority)
        query, params = '', dict()
        for ent in entities:
            ent_query, ent_params = ent.cypher()
            query += ent_query + '\n'
            params = params | ent_params
        return query, params
    
    def _nodes_relations_cypher(self, nodes: typing.Iterable[Node]) -> tuple[str, dict]:
        return self._entities_cypher(entities)
            
    @retry(tries=3, delay=5)
    def _add_batch(self, nodes: typing.Iterable[Node]) -> list:
        query, params = self._nodes_cypher(nodes)
        resp = self._conn.query(query, parameters=params)
        
        relations = [rel for node in nodes for rel in node.relations]
        for rel in relations:
            query, params = rel.cypher()
            resp += self._conn.query(query, parameters=params)
        return resp

    def _run(self, query, params):
        return self._conn.query(query, parameters=params)
    
    def _run_gc(self):
        logger.debug(f'collected {gc.collect()} objects with gc.')

#### test

In [123]:
# def test():
#     nn = build_test_nodes()
#     q, p = BaseGrapher(None)._nodes_cypher(nn)
#     print(p)
#     print(q)
#     print(p)
#     print(q)
# test()

## docs generation

In [125]:
class TextualExtractor(typing.Protocol):
    def __call__(self, text: str) -> typing.Iterable[Node]:
        pass

class DocsGrapher(BaseGrapher):
    def __init__(self,
                 conn, 
                 entities_manager,
                 text_splitter, 
                 extractors: list[TextualExtractor],
                 processors: list[Processor],
                 batch_size: int=1000,
            ):
        super().__init__(conn, batch_size)
        self.__extractors = extractors
        self.__processors = processors
        self.__text_splitter = text_splitter
        self.__entities_manager = entities_manager

    def __nodes_from_text(self, text: str) -> list[Node]:
        glance = text[:150].replace('\n', ' ')
        logger.debug(f'extracting span with text "{glance}..."')
        
        nodes = []
        for ext in self.__extractors:
            new_nodes = ext(text)
            logger.debug(f'got {len(new_nodes)} nodes from {ext}.')
            nodes += new_nodes

        self._run_gc()
        return nodes
        
    def nodes_from_doc(self, doc: Doc) -> tuple[list[Node], list[Node], list[Node]]:
        doc_node = self.__entities_manager.doc(doc)
        spans_docs = self.__text_splitter.create_documents([doc.full_text])
        logger.debug(f'got {len(spans_docs)} spans from doc.')
        
        spans_texts = [d.page_content for d in spans_docs]
        spans_nodes, kw_nodes = [], []
        for text in spans_texts:
            span_node = self.__entities_manager.span(text)
            span_kw_nodes = self.__nodes_from_text(text)
            for span_kw_node in span_kw_nodes:
                relation = self.__entities_manager.connect(
                    span_kw_node, span_node, RelationType.CONTAINS.value)
            spans_nodes.append(span_node)
            kw_nodes += span_kw_nodes
        self._run_gc()
        
        for span_node in spans_nodes:
            relation = self.__entities_manager.connect(
                span_node, doc_node, RelationType.PART_OF_TEXT.value)
        logger.debug(f'connecting doc to span nodes done.')
        return doc_node, spans_nodes, kw_nodes

    def nodes_from_docs(self, docs: typing.Iterable[Doc]) -> list[Node]:
        docs_nodes, spans_nodes, kw_nodes = [], [], []
        for i, doc in enumerate(docs):
            logger.info(f'extracting doc #{i+1}...')
            doc_node, doc_spans_nodes, doc_kw_nodes = self.nodes_from_doc(doc)
            logger.debug(f'got 1 doc node, {len(spans_nodes)} span nodes anb {len(kw_nodes)} kw nodes from doc.')
            docs_nodes.append(doc_node)
            spans_nodes += doc_spans_nodes
            kw_nodes += doc_kw_nodes
            
        
        nodes = docs_nodes + spans_nodes + kw_nodes
        for proc in self.__processors:
            nodes = proc(nodes)
            logger.debug(f'got {len(nodes)} nodes after {proc}.')
            
        logger.debug(f'final nodes deduplication done, got {len(nodes)}.')
        return nodes
    
    def add_entity(self, entity: Cypherable) -> list:
        query, params = entity.cypher()
        try:
            return self._conn.query(query, parameters=params)
        except Exception as e:
            logger.warning(f'bad node add, got {e} skipping...')
            return []
    
    def add_doc(self, doc: Doc) -> list:
        nodes = self.nodes_from_docs([doc])
        logger.debug(f'got {len(nodes)} nodes from doc.')
        entities = nodes + [rel for node in nodes for rel in node.relations]
        resp = []
        for ent in entities:
            query, params = ent.cypher()
            resp += self.add_entity(ent)
        return resp, nodes
    
    def add(self, docs: list[Doc]):
        resp, nodes = [], []
        for i, doc in enumerate(docs):
            r, n = self.add_doc(doc)
            logger.debug(f'doc {i} done. inserted batch with {len(n)} nodes.')
            resp += r
            nodes += n
        logger.debug(f'finaly got {len(nodes)} nodes from docs.')
        return resp, nodes

In [126]:
def doc_from_row(row):
    return Doc(title=row['title'], text=row['text'], link=row['link'])

docs = []
rows = articles_df

for i in range(rows.shape[0]):
    docs.append(doc_from_row(rows.iloc[i]))
display(rows)
print(len(docs))

,text,hash,title,link
0,Правильнее всего осматривать Юсуповский дворец...,4474475373476231224,Юсуповский дворец — дом богатейшей семьи Росс...,https://travel.mts.ru/media/guides/101-ideya/y...
1,Мечеть появилась в городе в 1913 году на деньг...,5723206605418886168,Соборная мечеть — мусульманский храм на Петро...,https://travel.mts.ru/media/guides/101-ideya/s...
2,Для обустройства своей летней резиденции будущ...,2641653856583702694,Павловский пейзажный парк — один из самых бол...,https://travel.mts.ru/media/guides/101-ideya/p...
3,Павловск Екатерина II подарила своему сыну Пав...,7660736790506138341,Большой дворец в Павловске: впечатляющая колле...,https://travel.mts.ru/media/guides/101-ideya/b...
4,В конструктивистском здании Научного городка с...,6608806741320959874,Музей «Павловские Колтуши» — первый российский...,https://travel.mts.ru/media/guides/101-ideya/m...
5,Сотни тысяч туристов по многу раз приезжают в ...,2305003458459233100,Апраксин двор — базар за имперскими фасадами,https://travel.mts.ru/media/guides/101-ideya/a...
6,Большинство зданий петербургского центра — это...,2385853299821716352,Толстовский дом и Дом Бенуа: знаменитые доходн...,https://travel.mts.ru/media/guides/101-ideya/t...
7,Зачем забираться под купол Исаакиевского собор...,2528798168100379865,Колоннада Исаакиевского собора: Вид с высоты н...,https://travel.mts.ru/media/guides/101-ideya/k...
8,Стрелкой в Петербурге называют мыс Васильевско...,2277906714354252759,Стрелка Васильевского острова: Главная петербу...,https://travel.mts.ru/media/guides/101-ideya/s...
9,Самый атмосферный книжный магазин в городе «По...,5217320806314180457,«Подписные издания» — самый атмосферный книжны...,https://travel.mts.ru/media/guides/101-ideya/p...


20


In [127]:
# entities_manager.doc(docs[0]).cypher()

### gen

In [250]:
print(gc.collect())

extractors = [
    spacy_extractor, 
    wikineural_extractor,
    mrebell_extractor,
]

processors = [
    disambiguator,
    stopwords_filter,
    size_filter,
    deduplicator,
] + embedders

grapher = DocsGrapher(
    conn, 
    entities_manager=entities_manager, 
    text_splitter=text_splitter,
    extractors=extractors,
    processors=processors,
)

def test():
    # q, p = grapher.cypher(docs[:1])
    # print(q)
    nodes = grapher.nodes_from_docs(docs)
    print(len(nodes))
    for v in nodes:
        if 'KeywordNode' in str(v.__class__):
            print(f'{v.text}, {len(v.relations)}')
    return nodes
            
# test()
# nodes = grapher.nodes_from_docs(docs)
# kw199, kw13, kw42, kw86
_, nodes = grapher.add(docs[10:])

108
extracting doc #1...
got 62 spans from doc.
extracting span with text "Маршрут по местам съемок фильма "Брат" в Петербурге | Санкт-Петербург Центр 17 мая 2017 года поклонники фильма "Брат" отметили 20-летгий юбилей выхода..."
got 5 nodes from <__main__.SpacyExtractor object at 0x2a9aa7610>.
got 7 nodes from <__main__.WikineuralExtractor object at 0x2a9a3ded0>.
extracting with MRebellExtractor...
MRebellExtractor tokenization done.
MRebellExtractor model work done.
MRebellExtractor decode done.
MRebellExtractor triplets decode done.
got 0 nodes from <__main__.MRebellExtractor object at 0x2a92e4690>.
collected 0 objects with gc.
extracting span with text "Фильм сразу же обрёл огромную популярность и искреннюю любовь среди зрителей нашей страны и за рубежом, а также небывалое признание и высокую оценку с..."
got 0 nodes from <__main__.SpacyExtractor object at 0x2a9aa7610>.
got 0 nodes from <__main__.WikineuralExtractor object at 0x2a9a3ded0>.
extracting with MRebellExtractor...
MRebel

/var/folders/st/brj0xbq51c9_tmkfz24p31dw0000gn/T/ipykernel_1460/4260229939.py:24: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  session = self.__driver.session(database=db) if db is not None else self.__driver.session()


MERGE (span1821:Span { embeddings: $span1821_embeddings, id: $span1821_id, text: $span1821_text })
ON MATCH SET span1821.matched = true, {'span1821_embeddings': [-0.1805008351802826, 0.015070142224431038, -0.10444329679012299, -0.33816418051719666, 0.020155394449830055, 0.11314237862825394, -0.007566933520138264, 0.1959359496831894, 0.1536012440919876, -0.016298504546284676, 0.0993356704711914, 0.17599812150001526, -0.14541155099868774, 0.38032492995262146, -0.3826504349708557, -0.09037154167890549, -0.003163344459608197, 0.03617975860834122, 0.1755995899438858, -0.15358182787895203, 0.05776058882474899, -0.11153384298086166, 0.22235792875289917, 0.1442718803882599, 0.17371532320976257, -0.09199842065572739, 0.20368841290473938, -0.02267688699066639, 0.05972420424222946, -0.09529872983694077, -0.12505538761615753, 0.03321622312068939, 0.3660467863082886, -0.025331225246191025, -0.09532782435417175, 0.08911719173192978, 0.000953516224399209, -0.30627205967903137, 0.04779446870088577, 0.

In [130]:
# _, nodes = grapher.add(docs[15:])

In [131]:
# %%time
# print_nodes(test())
# # CPU times: user 6min 23s, sys: 33.8 s, total: 6min 56s
# # Wall time: 55.8 s

In [132]:
# get_top_objects()

In [133]:
# gc.collect()
# gc.get_stats()
# gc.get_objects()

## places generation

In [135]:
class PlacesGrapher(BaseGrapher):
    def __init__(self, 
                 conn, 
                 entities_manager,
                 processors: list[Processor],
                 batch_size: int=1000,
            ):
        super().__init__(conn, batch_size)
        self.__entities_manager = entities_manager
        self.__processors = processors


    def nodes_from_place(self, place: Place) -> typing.Iterable[Node]:
        category_node = self.__entities_manager.keyword(place.category, 'CATEGORY')
        place_node = self.__entities_manager.place(place)
        city_node = self.__entities_manager.keyword(place.city, 'LOC')
        
        self.__entities_manager.connect(
            head=city_node,
            tail=place_node,
            type_=RelationType.LOCATED_IN_THE_ATE.value,
        )
        self.__entities_manager.connect(
            head=category_node,
            tail=place_node,
            type_=RelationType.HAS_CATEGORY.value,
        )
        
        return [category_node, place_node, city_node]
    
    def _cypher_from_nodes(self, nodes: typing.Iterable[Node]) -> tuple[str, dict]:
        entities = sorted(
            nodes + [rel for node in nodes for rel in node.relations], 
            key=lambda n: n.priority,
        )
        query, params = '', dict()
        for ent in entities:
            ent_query, ent_params = ent.cypher()
            query += ent_query + '\n'
            params = params | ent_params
        return query, params

    def nodes_from_places(self, places: typing.Iterable[Place]) -> typing.Iterable[Node]:
        nodes = [node for place in places for node in self.nodes_from_place(place)]
        logger.debug(f'got {len(nodes)} raw nodes.')
        
        for proc in self.__processors:
            nodes = proc(nodes)
            logger.debug(f'got {len(nodes)} nodes after {proc}.')
        
        logger.debug(f'final got {len(nodes)}.')
        return nodes

    def cypher(self, places: typing.Iterable[Place]) -> tuple[str, dict]:
        nodes = self.nodes_from_places(places)
        logger.debug(f'finaly got {len(nodes)} nodes from places.')
        return self.cypher_from_nodes(nodes)
    
    def add_batch(self, batch: typing.Iterable[Node]) -> tuple[list, list]:
        nodes = self.nodes_from_places(batch)
        logger.debug(f'finaly got {len(nodes)} nodes from batch.')
        resp = self._add_batch(nodes)
        logger.debug(f'inserted batch with {len(nodes)} nodes.')
        return resp, nodes
    
    def add(self, places: typing.Iterable[Place]) -> list:
        resp, nodes = [], []
        for batch in self._make_batches(places, 50):
            r, n = self.add_batch(batch)
            resp += r
            nodes += n
        return resp, nodes

### gen

In [137]:
places_df = pd.read_csv('../../data/places4nodes.csv', sep=',')
places_df = places_df[places_df['city'] == 'Санкт-Петербург'].copy()
places_df.keys(), places_df.shape

(Index(['address', 'category', 'city', 'lat', 'lon', 'rating', 'region',
        'region_code', 'subcategory', 'title', 'object_keys_text', 'is_modern',
        'has_music', 'fill_score', 'is_accessable', 'review_count',
        'worktime.is_24x7', 'has_wifi', 'contact_social_len', 'phone_len',
        'photos_len', 'has_website', 'biased_rating'],
       dtype='object'),
 (14678, 23))

In [138]:
places_df.isna().any()

address               False
category              False
city                  False
lat                   False
lon                   False
rating                False
region                False
region_code           False
subcategory            True
title                 False
object_keys_text      False
is_modern             False
has_music             False
fill_score            False
is_accessable         False
review_count          False
worktime.is_24x7      False
has_wifi              False
contact_social_len    False
phone_len             False
photos_len            False
has_website           False
biased_rating         False
dtype: bool

In [139]:
places_df['subcategory'] = places_df['subcategory'].fillna('Unknown')

In [140]:
def place_from_row(row):
    return Place(
        text = row['object_keys_text'],
        address = row['address'],
        category = row['category'],
        city = row['city'],
        lat = row['lat'],
        lon = row['lon'],
        rating = row['rating'],
        subcategory = row['subcategory'],
        title = row['title'],
        object_keys_text = row['object_keys_text'],
        is_modern = row['is_modern'],
        has_music = row['has_music'],
        fill_score = row['fill_score'],
        is_accessable = row['is_accessable'],
        review_count = row['review_count'],
        worktime_is_24x7 = row['worktime.is_24x7'],
        has_wifi = row['has_wifi'],
        contact_social_len = row['contact_social_len'],
        phone_len = row['phone_len'],
        photos_len = row['photos_len'],
        has_website = row['has_website'],
        biased_rating = row['biased_rating'],
    )

def sample(df, n=50):
    places = []
    for c in df['category'].unique():
        if c in ['services']:
            continue
        category_df = df[df['category'] == c]
        if len(category_df) > n:
            category_df = category_df.sample(n)
        places += list(category_df.apply(place_from_row, axis=1))
        print(c, category_df.shape[0])
    return places

places = sample(places_df[50:], 42)
len(places)

recreation 42
entertainment 42
sports 18
shopping 42
religious 42
food 42


228

In [141]:
processors = [
    disambiguator,
    size_filter,
    stopwords_filter,
    deduplicator,
] + embedders

grapher = PlacesGrapher(
    conn, 
    entities_manager, 
    processors,
)
# q, p = grapher.cypher(places)
# print(q)
# nodes = grapher.nodes_from_places(places)
# print_nodes(nodes)
r, n = grapher.add(places)

got 150 raw nodes.
got 150 nodes after <__main__.BasicDisambiguator object at 0x2a9a84690>.
got 150 nodes after <__main__.SizeFilter object at 0x2a9a8b090>.
got 150 nodes after <__main__.StopwordsFilter object at 0x2a9a96250>.
got 53 nodes after <__main__.BasicDeduplicator object at 0x2a9a889d0>.
got 53 nodes after <__main__.DocsEmbedder object at 0x2aa5f2b90>.
got 53 nodes after <__main__.SpansEmbedder object at 0x2aa2dd050>.
got 53 nodes after <__main__.PlacesEmbedder object at 0x2aa60ec90>.
final got 53.
finaly got 53 nodes from batch.
MERGE (kw680:Keyword { type_: $kw680_type_, text: $kw680_text })
ON MATCH SET kw680.matched = true
MERGE (place681:Place { embeddings: $place681_embeddings, id: $place681_id, address: $place681_address, category: $place681_category, city: $place681_city, lat: $place681_lat, lon: $place681_lon, rating: $place681_rating, subcategory: $place681_subcategory, title: $place681_title, object_keys_text: $place681_object_keys_text, is_modern: $place681_is_mode

# Embeddings

In [143]:
conn.query('''
CREATE VECTOR INDEX `doc_embeddings` IF NOT EXISTS
FOR (n: Doc) ON (n.embeddings)
OPTIONS {indexConfig: {
 `vector.dimensions`: 384,
 `vector.similarity_function`: 'cosine'
}}
''')

conn.query('''
CREATE VECTOR INDEX `span_embeddings` IF NOT EXISTS
FOR (n: Span) ON (n.embeddings)
OPTIONS {indexConfig: {
 `vector.dimensions`: 384,
 `vector.similarity_function`: 'cosine'
}}
''')

conn.query('''
CREATE VECTOR INDEX `place_embeddings` IF NOT EXISTS
FOR (n: Place) ON (n.embeddings)
OPTIONS {indexConfig: {
 `vector.dimensions`: 384,
 `vector.similarity_function`: 'cosine'
}}
''')


CREATE VECTOR INDEX `doc_embeddings` IF NOT EXISTS
FOR (n: Doc) ON (n.embeddings)
OPTIONS {indexConfig: {
 `vector.dimensions`: 384,
 `vector.similarity_function`: 'cosine'
}}
, None

CREATE VECTOR INDEX `span_embeddings` IF NOT EXISTS
FOR (n: Span) ON (n.embeddings)
OPTIONS {indexConfig: {
 `vector.dimensions`: 384,
 `vector.similarity_function`: 'cosine'
}}
, None

CREATE VECTOR INDEX `place_embeddings` IF NOT EXISTS
FOR (n: Place) ON (n.embeddings)
OPTIONS {indexConfig: {
 `vector.dimensions`: 384,
 `vector.similarity_function`: 'cosine'
}}
, None


[]

In [144]:
# conn.query('DROP INDEX doc_embeddings')
# conn.query('DROP INDEX span_embeddings')
# conn.query('DROP INDEX place_embeddings')

In [145]:
# асскажи про  Стрелку Васильевского острова

In [146]:
# # title = 'Юсуповский дворец — дом богатейшей семьи Российской империи и место убийства Григория Распутина'
# title = 'асскажи про  Стрелку Васильевского острова'
# test_embeds = miniLM_embedder(title)
# test_embeds[:3]

In [147]:
# index_query = '''
# CALL db.index.vector.queryNodes('doc_embeddings', $limit, $embeddings) 
# YIELD node, score
# MATCH (node)
# WHERE score > 0.6
# RETURN 
#     node.title, 
#     score
# ORDER BY score DESC 
# '''

# select_query = '''
# '''

# query_text = index_query
# # query_text = select_query

# print(query_text)
# conn.query(query_text, parameters={
#     'embeddings': test_embeds,
#     'limit': 3,
# })#, test_embeds

In [148]:
conn.close()